In [39]:
setwd(paste0(Sys.getenv('R_SOURCES'), '/house_prices'))
getwd()

[1] "/mnt/hdd/develop/kaggle/R_sources/house_prices"

In [59]:
source('main.R')
house_prices$helpers$import_libs()

combined_dataset <- house_prices$helpers$load_data()

combined_dataset_fixed <-
    combined_dataset %>%
    (house_prices$missing$categ$replace_with_most_common) %>%
    (house_prices$missing$categ$fix_valid) %>%
    (house_prices$missing$numeric$replace_with_zero)

training_dataset <- combined_dataset_fixed %>% filter(dataSource == 'train')


test_that("should be no NA values in training_dataset", {
    expect_equal(sum(is.na(training_dataset)), 0)
})

In [61]:
categ_data <- 
    training_dataset %>%
    # select categ vars
    select(house_prices$helpers$get_character_colnames(combined_dataset_fixed), SalePrice, -dataSource) %>%
    # drop weak categ vars
    select(-one_of(house_prices$attributes_selection$discared_cat_vars)) %>%
    mutate(
        price_log = log(SalePrice)
    ) %>%
    select(-SalePrice)

categ_data %>% colnames %>% sort

[1] "Alley"         "BldgType"      "BsmtCond"      "BsmtExposure" 
 [5] "BsmtFinType1"  "BsmtQual"      "CentralAir"    "Electrical"   
 [9] "Exterior1st"   "Exterior2nd"   "ExterQual"     "Fence"        
[13] "FireplaceQu"   "Foundation"    "GarageCond"    "GarageFinish" 
[17] "GarageQual"    "GarageType"    "HeatingQC"     "HouseStyle"   
[21] "KitchenQual"   "LandContour"   "LotConfig"     "LotShape"     
[25] "MasVnrType"    "MiscFeature"   "MSSubClass"    "MSZoning"     
[29] "Neighborhood"  "PavedDrive"    "PoolQC"        "price_log"    
[33] "RoofStyle"     "SaleCondition" "SaleType"

In [103]:
categ_data %>%
    select(price_log) %>%
    summarise(
        q25 = quantile(price_log, 0.25),
        q50 = quantile(price_log, 0.5),
        q75 = quantile(price_log, 0.75)
    ) -> global_quantiles
global_quantiles

q25,q50,q75
11.7751,12.00151,12.27373


In [120]:
categ_data %>%
gather(var, value, -price_log) %>%
group_by(var, value) %>%
nest %>%
mutate(
    dist = map(data, ~ecdf(.$price_log)),
    rating = map_dbl(dist, function (cdf) {
        prob_rating_1 <- cdf(global_quantiles$q25)
        prob_rating_2 <- cdf(global_quantiles$q50) - cdf(global_quantiles$q25)
        prob_rating_3 <- cdf(global_quantiles$q75) - cdf(global_quantiles$q50)
        prob_rating_4 <- 1 - cdf(global_quantiles$q75)
        group_rating <- prob_rating_1 * 1 + prob_rating_2 * 2 + prob_rating_3 * 3 + prob_rating_4 * 4
        group_rating
    })
) %>%
select(-dist) %>% 
unnest -> xxx

[1] 0.2403214

[1] 0.2490869

[1] 0.2490869

[1] 0.2615047

[1] 2.531775